# STW7071CEM (Information Retrieval) Assignment

Task 1:

Create a vertical search engine comparable to Google Scholar, but specialized in retrieving just papers/books published by a member of Coventry University's Research Centre for Intelligent Healthcare (RCIH):


## Package Installations 


In [7]:
!pip install scrapy
!pip install requests
!pip install BeautifulSoup4
!pip install nltk
!pip install gensim
!pip install xgboost
!pip install pandastable

## Importing Packages 


In [8]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import time
import datetime
import string
import json
import nltk
nltk.download('omw-1.4');
nltk.download('averaged_perceptron_tagger')
nltk.download('wordnet')
nltk.download('stopwords')
nltk.download('punkt')
from nltk import pos_tag
from nltk.corpus import wordnet
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer 

[nltk_data] Downloading package omw-1.4 to /Users/kule/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /Users/kule/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package wordnet to /Users/kule/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to /Users/kule/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /Users/kule/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


# 1. Crawler Component

In [9]:
URL = "https://pureportal.coventry.ac.uk/en/organisations/coventry-university/persons/"
profile_url = "https://pureportal.coventry.ac.uk/en/persons/"

In [10]:
def maximum_page():
    
    first = requests.get(URL)
    soup = BeautifulSoup(first.text, 'html.parser')
    final_page = soup.select('#main-content > div > section > nav > ul > li:nth-child(12) > a')[0]['href']
    fp = final_page.split('=')[-1]
    return int(fp)
    
mx = maximum_page()
mx

46

Instead of crawling all researchers, this web crawler is designed to specifically find researchers who:
    
1. Have research publications
2. Is a part of Coventry University's Research Centre for Intelligent Healthcare

In [11]:
def check_department(researcher):
    
    l1 = researcher.find('div', class_='rendering_person_short')
      
    for span in l1.find_all('span'):
        # Check department
        #print(span.text)
        if span.text == str('Centre for Intelligent Healthcare'):
            name = researcher.find('h3', class_='title').find('span').text
            return name
        else:
            pass

def create_csv():
     database = pd.DataFrame(columns=['Title', 'Author', 'Published', 'Link'])
     database.to_csv('database.csv')
    
def update_csv(database):
    current_data = pd.read_csv(database, index_col="Unnamed: 0")
    return current_data        

def enter_each_researchers_publication(researcher, url, df):

    new_url = url + str(researcher).replace(' ','-').lower() + '/publications/'
    page = requests.get(new_url)
    soup = BeautifulSoup(page.content, "html.parser")
    results = soup.find(id="main-content")
    papers = results.find_all("li", class_="list-result-item")

    for paper in papers:
        title = paper.find('h3', class_='title')
        if title is not None:
            title_span = title.find('span')
            title_text = title_span.text if title_span is not None else "N/A"
        else:
            title_text = "N/A"

        author = paper.find('a', class_='link person')
        print(author)
        if author == None or author == 'None':
            continue
        else:
            author_span = author.find('span')
            author_text = author_span.text if author_span is not None else "N/A"

        date = paper.find('span', class_="date")
        date_text = date.text if date is not None else "N/A"

        link = paper.find('h3', class_='title')
        link_href = link.find('a', href=True)['href'] if link is not None else "N/A"
        print(link_href)

        # Read the existing database.csv file
        opening = pd.read_csv('database.csv', index_col="Unnamed: 0")

        # Create a new DataFrame with the data to be appended
        new_row = pd.DataFrame({'Title': [title.text],
                                'Author': [author.text],
                                'Published': [date.text],
                                'Link': [link_href]})

        # Concatenate the existing DataFrame and the new row DataFrame
        opening = pd.concat([opening, new_row], ignore_index=True)

        # Save the updated DataFrame to database.csv
        opening.to_csv('database.csv')

                

In [12]:
def scrape(mx):
    df = update_csv('database.csv')
    i=0
    while True:
    
        if i > 17:
            break
            
        if i>0:
            url = URL + '?page=' + str(i)
        else:
            url = URL
    
        i = i+1
        # scraping starts here
        page = requests.get(url)
        soup = BeautifulSoup(page.content, "html.parser")
        results = soup.find(id="main-content")
        researchers = results.find_all("li", class_="grid-result-item")

        for researcher in researchers:
            # Check if researcher has any papers
            check = researcher.find('div', class_='stacked-trend-widget')
            if check:
                name = check_department(researcher)
                if name is None:
                    pass
                else:
                    enter_each_researchers_publication(name, profile_url, df)

In [13]:
create_csv()
update_csv(database='database.csv') #create_csv

%time scrape(mx) 

<a class="link person" href="https://pureportal.coventry.ac.uk/en/persons/john-allen" rel="Person"><span>Allen, J.</span></a>
https://pureportal.coventry.ac.uk/en/publications/deep-learning-identification-of-coronary-artery-disease-from-bila
<a class="link person" href="https://pureportal.coventry.ac.uk/en/persons/kim-bul" rel="Person"><span>Bul, K.</span></a>
https://pureportal.coventry.ac.uk/en/publications/usability-and-preliminary-efficacy-of-an-ai-driven-platform-suppo-2
<a class="link person" href="https://pureportal.coventry.ac.uk/en/persons/john-allen" rel="Person"><span>Allen, J.</span></a>
https://pureportal.coventry.ac.uk/en/publications/assessing-hemodynamics-from-the-photoplethysmogram-to-gain-insigh
<a class="link person" href="https://pureportal.coventry.ac.uk/en/persons/john-allen" rel="Person"><span>Allen, J.</span></a>
https://pureportal.coventry.ac.uk/en/publications/book-pitch-article-on-photoplethysmography-technology-signal-anal
<a class="link person" href="https:

<a class="link person" href="https://pureportal.coventry.ac.uk/en/persons/richard-aspinall" rel="Person"><span>Aspinall, R.</span></a>
https://pureportal.coventry.ac.uk/en/publications/active-hhv-6-infection-of-cerebellar-purkinje-cells-in-mood-disor
<a class="link person" href="https://pureportal.coventry.ac.uk/en/persons/richard-aspinall" rel="Person"><span>Aspinall, R.</span></a>
https://pureportal.coventry.ac.uk/en/publications/interventions-to-restore-appropriate-immune-function-in-the-elder
<a class="link person" href="https://pureportal.coventry.ac.uk/en/persons/richard-aspinall" rel="Person"><span>Aspinall, R.</span></a>
https://pureportal.coventry.ac.uk/en/publications/perturbation-of-the-t-cell-receptor-repertoire-occurs-with-increa
<a class="link person" href="https://pureportal.coventry.ac.uk/en/persons/richard-aspinall" rel="Person"><span>Aspinall, R.</span></a>
https://pureportal.coventry.ac.uk/en/publications/polymorphisms-in-the-canine-il7r-3utr-are-associated-with-thym

<a class="link person" href="https://pureportal.coventry.ac.uk/en/persons/kim-bul" rel="Person"><span>Bul, K.</span></a>
https://pureportal.coventry.ac.uk/en/publications/a-serious-game-for-patients-with-eating-disorders-maze-out-pilot-
<a class="link person" href="https://pureportal.coventry.ac.uk/en/persons/kim-bul" rel="Person"><span>Bul, K.</span></a>
https://pureportal.coventry.ac.uk/en/publications/can-ehealth-applications-improve-renal-transplant-outcomes-for-ad
<a class="link person" href="https://pureportal.coventry.ac.uk/en/persons/kim-bul" rel="Person"><span>Bul, K.</span></a>
https://pureportal.coventry.ac.uk/en/publications/perceived-support-from-best-friends-and-depressive-symptoms-durin
<a class="link person" href="https://pureportal.coventry.ac.uk/en/persons/kim-bul" rel="Person"><span>Bul, K.</span></a>
https://pureportal.coventry.ac.uk/en/publications/the-moderating-role-of-co-rumination-on-the-association-between-f
<a class="link person" href="https://pureportal.cove

In [14]:
sample_db = pd.read_csv('database.csv').rename(columns={'Unnamed: 0':'SN'})
sample_db
print(f'{sample_db.shape[0]} records were scraped')

123 records were scraped


# 2. Indexing 

In [15]:
scraped_db = pd.read_csv('database.csv').rename(columns={'Unnamed: 0':'SN'}).reset_index(drop=True)
scraped_db.head()
# scraped_db = pd.read_csv('database.csv', index_col=0)

,SN,Title,Author,Published,Link
0,0,Deep learning identification of coronary arter...,"Allen, J.",1 Sept 2023,https://pureportal.coventry.ac.uk/en/publicati...
1,1,Usability and preliminary efficacy of an AI-dr...,"Bul, K.",16 Apr 2023,https://pureportal.coventry.ac.uk/en/publicati...
2,2,Assessing hemodynamics from the photoplethysmo...,"Allen, J.",1 Apr 2022,https://pureportal.coventry.ac.uk/en/publicati...
3,3,Book Pitch article on Photoplethysmography: te...,"Allen, J.",25 May 2022,https://pureportal.coventry.ac.uk/en/publicati...
4,4,Ethnic Disparities in Publicly-available Pulse...,"Allen, J.",27 May 2022,https://pureportal.coventry.ac.uk/en/publicati...


In [16]:
sample_db.head(7)
#ids = scraped_db["Title"]
#scraped_db[ids.isin(ids[ids.duplicated()])]

,SN,Title,Author,Published,Link
0,0,Deep learning identification of coronary arter...,"Allen, J.",1 Sept 2023,https://pureportal.coventry.ac.uk/en/publicati...
1,1,Usability and preliminary efficacy of an AI-dr...,"Bul, K.",16 Apr 2023,https://pureportal.coventry.ac.uk/en/publicati...
2,2,Assessing hemodynamics from the photoplethysmo...,"Allen, J.",1 Apr 2022,https://pureportal.coventry.ac.uk/en/publicati...
3,3,Book Pitch article on Photoplethysmography: te...,"Allen, J.",25 May 2022,https://pureportal.coventry.ac.uk/en/publicati...
4,4,Ethnic Disparities in Publicly-available Pulse...,"Allen, J.",27 May 2022,https://pureportal.coventry.ac.uk/en/publicati...
5,5,Novel pulse device for diagnosis of peripheral...,"Allen, J.",24 Oct 2022,https://pureportal.coventry.ac.uk/en/publicati...
6,6,Prospective assessment of the diagnostic accur...,"Allen, J.",18 Aug 2022,https://pureportal.coventry.ac.uk/en/publicati...


In [17]:
single_row = scraped_db.loc[1,:].copy()
single_row

SN                                                           1
Title        Usability and preliminary efficacy of an AI-dr...
Author                                                 Bul, K.
Published                                          16 Apr 2023
Link         https://pureportal.coventry.ac.uk/en/publicati...
Name: 1, dtype: object

## 2.1 Preprocessing Text

In [18]:
# Removing stop words
sw = stopwords.words("english")
lemmatizer = WordNetLemmatizer()

def tp1(txt):
    txt = txt.lower()   # Make lowercase
    txt = txt.translate(str.maketrans('','',string.punctuation))   # Remove punctuation marks
    txt = lematize(txt)
    return txt


def fwpt(word):
    tag = pos_tag([word])[0][1][0].upper()
    hash_tag = {"V": wordnet.VERB, "R": wordnet.ADV,"N": wordnet.NOUN,"J": wordnet.ADJ}         
    return hash_tag.get(tag, wordnet.NOUN)

def lematize(text):
        tkns = nltk.word_tokenize(text)
        ax = ""
        for each in tkns:
            if each not in sw:
                ax += lemmatizer.lemmatize(each, fwpt(each)) + " "
        return ax


In [19]:
# Sample title
single_row['Title']

'Usability and preliminary efficacy of an AI-driven platform supporting dietary management in diabetes: A mixed-method study'

In [20]:
# Demonstration of lowercase and punctuation removal
tp1(single_row['Title'])

'usability preliminary efficacy aidriven platform support dietary management diabetes mixedmethod study '

In [21]:
# Demonstration of lematization

lematize(tp1(single_row['Title']))
#lematize(single_row['Title'])

'usability preliminary efficacy aidriven platform support dietary management diabetes mixedmethod study '

### 2.1.1 Preprocess entire dataframe

In [22]:
processed_db = scraped_db.copy()

def preprocess_df(df):
    df.Title = df.Title.apply(tp1)
    df.Author = df.Author.str.lower()
    df = df.drop(columns=['Author','Published'], axis=1)
    return df
    
preprocess_df(processed_db)
processed_db.head()

,SN,Title,Author,Published,Link
0,0,deep learn identification coronary artery dise...,"allen, j.",1 Sept 2023,https://pureportal.coventry.ac.uk/en/publicati...
1,1,usability preliminary efficacy aidriven platfo...,"bul, k.",16 Apr 2023,https://pureportal.coventry.ac.uk/en/publicati...
2,2,assess hemodynamics photoplethysmogram gain in...,"allen, j.",1 Apr 2022,https://pureportal.coventry.ac.uk/en/publicati...
3,3,book pitch article photoplethysmography techno...,"allen, j.",25 May 2022,https://pureportal.coventry.ac.uk/en/publicati...
4,4,ethnic disparity publiclyavailable pulse oxime...,"allen, j.",27 May 2022,https://pureportal.coventry.ac.uk/en/publicati...


## 2.2 Index Construction

In [23]:
single = processed_db.loc[0,:].copy()
print(single)
indexing_trial = {}

words = single.Title.split()
SN = single.SN
word = words[0]
example = {word: [SN]}

print('=====================================================================')
print('Sample index')
print(example)

SN                                                           0
Title        deep learn identification coronary artery dise...
Author                                               allen, j.
Published                                          1 Sept 2023
Link         https://pureportal.coventry.ac.uk/en/publicati...
Name: 0, dtype: object
Sample index
{'deep': [0]}


In [24]:
## Indexer Function
def apply_index(inputs, index):
    words = inputs.Title.split()
    SN = int(inputs.SN)
    for word in words:
        if word in index.keys():
            if SN not in index[word]:
                index[word].append(SN)
        else:
            index[word] = [SN]
    return index

indx = apply_index(inputs=single, index= {})
#print(indx)

In [25]:
def full_index(df, index):
    for x in range(len(df)):
        inpt = df.loc[x,:]
        ind = apply_index(inputs=inpt, index=index)
    return ind

def construct_index(df, index):
    queue = preprocess_df(df)
    ind = full_index(df=queue, index=index)
    return ind

indexed = full_index(processed_db, 
                     index = {})


indexes = construct_index(df=scraped_db, 
                          index = {})

In [26]:
with open('indexes.json', 'w') as new_f:
    json.dump(indexes, new_f, sort_keys=True, indent=4)
    
with open('indexes.json', 'r') as file:
    data = json.load(file)

def index_2(df, x_path):
    if len(df) > 0:
        with open(x_path, 'r') as file:
            prior_index = json.load(file)
        new_index = construct_index(df = df, index = prior_index)
        with open(x_path, 'w') as new_f:
            json.dump(new_index, new_f, sort_keys=True, indent=4)

In [27]:
len(data)

612

In [28]:
data

{'185': [52],
 '21st': [94],
 '3rs': [80],
 '3′utr': [52],
 '7': [84],
 'aankomende': [106],
 'abnormality': [31],
 'acceptance': [98],
 'accord': [59],
 'accuracy': [6, 20],
 'across': [107],
 'activation': [64],
 'active': [49],
 'adaptive': [61],
 'address': [111, 113],
 'adhd': [119, 120],
 'adjunct': [115],
 'adolescence': [100],
 'adolescent': [99, 101, 107, 114],
 'adult': [39, 77, 83, 85, 94, 99],
 'advance': [21],
 'age': [2, 19, 51, 54, 62, 63, 77, 79, 91, 93],
 'ageassociated': [59, 96],
 'agerelated': [22, 90],
 'aidriven': [1, 7, 103, 104],
 'al': [105],
 'amplitude': [41],
 'analysis': [3, 13, 41, 42, 44, 55, 56, 77, 102],
 'animal': [69],
 'anklebrachial': [33],
 'antigrippes': [78],
 'anxiety': [102],
 'application': [3, 13, 14, 99],
 'apply': [44],
 'approach': [26, 54, 114],
 'appropriate': [50],
 'arterial': [5, 8, 9, 20, 33, 34, 38, 40, 41, 42, 46],
 'artery': [0, 6],
 'article': [3],
 'as': [105],
 'assay': [85],
 'assess': [2],
 'assessment': [6, 12, 15, 23, 30, 3

## 3.  Query Processor

In [29]:
def demonstrate_query_processing():
    sample = input('Enter Search Terms: ')
    processed_query = tp1(sample)
    #print(f'User Search Query: {sample}')
    print(f'Processed Search Query: {processed_query}')
    return processed_query
    
#demonstrate_query_processing()

### 3.1.  Split Query into individual terms

In [30]:
def split_query(terms):
    each = tp1(terms)
    return each.split()

dqp = demonstrate_query_processing()
dqp
print(f'Split Query: {split_query(dqp)}')

Enter Search Terms: 
Processed Search Query: 
Split Query: []


### 3.2.  Boolean Functionalities

In [31]:
def union(lists):
    union = list(set.union(*map(set, lists)))
    union.sort()
    return union

def intersection(lists):
    intersect = list(set.intersection(*map(set, lists)))
    intersect.sort()
    return intersect

### 3.3. Search Engine Function

In [32]:
def vertical_search_engine(df, query, index=indexes):
    query_split = split_query(query)
    retrieved = []
    for word in query_split:
        if word in index.keys():
            retrieved.append(index[word])
            
            
    # Ranked Retrieval
    if len(retrieved)>0:
        high_rank_result = intersection(retrieved)
        low_rank_result = union(retrieved) 
        c = [x for x in low_rank_result if x not in high_rank_result]      
        high_rank_result.extend(c)
        result = high_rank_result
        
        final_output = df[df.SN.isin(result)].reset_index(drop=True)
    
        # Return result in order of Intersection ----> Union
        dummy = pd.Series(result, name = 'SN').to_frame()
        result = pd.merge(dummy, final_output, on='SN', how = 'left')
        
    else:
        result = 'No result found'
    
    return result

In [33]:
def test_search_engine():
    xtest = scraped_db.copy()
    query = input("Enter your search query: ")
    return vertical_search_engine(xtest, query, indexed)
    
test_search_engine()

Enter your search query: 


'No result found'

In [34]:
def final_engine(results):
    if type(results) != 'list':
        return results
        #print(results)
    else:
        for i in range(len(results)):
            printout = results.loc[i, :]

In [35]:
scraped_db['Author'].iloc[24]

'zheng, d.'

In [36]:
final_engine(test_search_engine())

Enter your search query: 


'No result found'

## 4. Schedule Crawler for every week

To demonstrate a weekly scheduled crawling, the following parameters are defined:

* `interval` : Represents number of days in reality. In this code, it represents only seconds for demonstration


In [37]:
days = 0
interval = 7
while days <= 1:
    scrape(mx)
    print(f"Crawled at {datetime.datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")
    print(f'Next crawl scheduled after {interval} days')
    time.sleep(interval)
    days = days + 1

<a class="link person" href="https://pureportal.coventry.ac.uk/en/persons/john-allen" rel="Person"><span>Allen, J.</span></a>
https://pureportal.coventry.ac.uk/en/publications/deep-learning-identification-of-coronary-artery-disease-from-bila
<a class="link person" href="https://pureportal.coventry.ac.uk/en/persons/kim-bul" rel="Person"><span>Bul, K.</span></a>
https://pureportal.coventry.ac.uk/en/publications/usability-and-preliminary-efficacy-of-an-ai-driven-platform-suppo-2
<a class="link person" href="https://pureportal.coventry.ac.uk/en/persons/john-allen" rel="Person"><span>Allen, J.</span></a>
https://pureportal.coventry.ac.uk/en/publications/assessing-hemodynamics-from-the-photoplethysmogram-to-gain-insigh
<a class="link person" href="https://pureportal.coventry.ac.uk/en/persons/john-allen" rel="Person"><span>Allen, J.</span></a>
https://pureportal.coventry.ac.uk/en/publications/book-pitch-article-on-photoplethysmography-technology-signal-anal
<a class="link person" href="https:

<a class="link person" href="https://pureportal.coventry.ac.uk/en/persons/richard-aspinall" rel="Person"><span>Aspinall, R.</span></a>
https://pureportal.coventry.ac.uk/en/publications/active-hhv-6-infection-of-cerebellar-purkinje-cells-in-mood-disor
<a class="link person" href="https://pureportal.coventry.ac.uk/en/persons/richard-aspinall" rel="Person"><span>Aspinall, R.</span></a>
https://pureportal.coventry.ac.uk/en/publications/interventions-to-restore-appropriate-immune-function-in-the-elder
<a class="link person" href="https://pureportal.coventry.ac.uk/en/persons/richard-aspinall" rel="Person"><span>Aspinall, R.</span></a>
https://pureportal.coventry.ac.uk/en/publications/perturbation-of-the-t-cell-receptor-repertoire-occurs-with-increa
<a class="link person" href="https://pureportal.coventry.ac.uk/en/persons/richard-aspinall" rel="Person"><span>Aspinall, R.</span></a>
https://pureportal.coventry.ac.uk/en/publications/polymorphisms-in-the-canine-il7r-3utr-are-associated-with-thym

<a class="link person" href="https://pureportal.coventry.ac.uk/en/persons/kim-bul" rel="Person"><span>Bul, K.</span></a>
https://pureportal.coventry.ac.uk/en/publications/a-serious-game-for-patients-with-eating-disorders-maze-out-pilot-
<a class="link person" href="https://pureportal.coventry.ac.uk/en/persons/kim-bul" rel="Person"><span>Bul, K.</span></a>
https://pureportal.coventry.ac.uk/en/publications/can-ehealth-applications-improve-renal-transplant-outcomes-for-ad
<a class="link person" href="https://pureportal.coventry.ac.uk/en/persons/kim-bul" rel="Person"><span>Bul, K.</span></a>
https://pureportal.coventry.ac.uk/en/publications/perceived-support-from-best-friends-and-depressive-symptoms-durin
<a class="link person" href="https://pureportal.coventry.ac.uk/en/persons/kim-bul" rel="Person"><span>Bul, K.</span></a>
https://pureportal.coventry.ac.uk/en/publications/the-moderating-role-of-co-rumination-on-the-association-between-f
<a class="link person" href="https://pureportal.cove

<a class="link person" href="https://pureportal.coventry.ac.uk/en/persons/richard-aspinall" rel="Person"><span>Aspinall, R.</span></a>
https://pureportal.coventry.ac.uk/en/publications/active-hhv-6-infection-of-cerebellar-purkinje-cells-in-mood-disor
<a class="link person" href="https://pureportal.coventry.ac.uk/en/persons/richard-aspinall" rel="Person"><span>Aspinall, R.</span></a>
https://pureportal.coventry.ac.uk/en/publications/interventions-to-restore-appropriate-immune-function-in-the-elder
<a class="link person" href="https://pureportal.coventry.ac.uk/en/persons/richard-aspinall" rel="Person"><span>Aspinall, R.</span></a>
https://pureportal.coventry.ac.uk/en/publications/perturbation-of-the-t-cell-receptor-repertoire-occurs-with-increa
<a class="link person" href="https://pureportal.coventry.ac.uk/en/persons/richard-aspinall" rel="Person"><span>Aspinall, R.</span></a>
https://pureportal.coventry.ac.uk/en/publications/polymorphisms-in-the-canine-il7r-3utr-are-associated-with-thym

<a class="link person" href="https://pureportal.coventry.ac.uk/en/persons/kim-bul" rel="Person"><span>Bul, K.</span></a>
https://pureportal.coventry.ac.uk/en/publications/a-serious-game-for-patients-with-eating-disorders-maze-out-pilot-
<a class="link person" href="https://pureportal.coventry.ac.uk/en/persons/kim-bul" rel="Person"><span>Bul, K.</span></a>
https://pureportal.coventry.ac.uk/en/publications/can-ehealth-applications-improve-renal-transplant-outcomes-for-ad
<a class="link person" href="https://pureportal.coventry.ac.uk/en/persons/kim-bul" rel="Person"><span>Bul, K.</span></a>
https://pureportal.coventry.ac.uk/en/publications/perceived-support-from-best-friends-and-depressive-symptoms-durin
<a class="link person" href="https://pureportal.coventry.ac.uk/en/persons/kim-bul" rel="Person"><span>Bul, K.</span></a>
https://pureportal.coventry.ac.uk/en/publications/the-moderating-role-of-co-rumination-on-the-association-between-f
<a class="link person" href="https://pureportal.cove

# 5. GUI

In [ ]:
from tkinter import *
from tkinter import messagebox
from PIL import Image, ImageTk 
from tkinter import scrolledtext
from pandastable import Table, TableModel
from contextlib import suppress
import warnings
warnings.filterwarnings('ignore')



image1 = Image.open('coventry-university-logo.png')
resized_image1 = image1.resize((250,200), Image.ANTIALIAS)

def new_gui(image1):
    window = Tk()
    window.configure(bg='#F0F0F0')
    window.title("CU Search Engine")
    window.geometry('900x700')
    
    lbl = Label(window, text="Search Engine",bg="#F0F0F0", font=("Calibri", 30), padx=5, pady=5)
    lbl.grid(column=1, row=1)
    
    lbl2 = Label(window, text="Enter the search query here", bg="#F0F0F0",font=("Calibri", 15), padx=5, pady=5)
    lbl2.grid(column=0, row=2)
    
    
    img = ImageTk.PhotoImage(image1)
    
    lbl3 = Label(image=img)
    lbl3.image = img
    lbl3.grid(column=3, row=5, padx=5, pady=5)
    
    
    
    query = Entry(window,width=80)
    query.grid(column=1, row=2,  padx=5, pady=5)
    
    results = Canvas(window, height=40, width=350)
    results.grid(column=1, row=3, padx=5, pady=5)
    
    # Entry
    def getInputBoxValue():
        userInput = query.get()
        return userInput

    
    # Button
    def clicked():
        search()
        #pass
        
    def no_result():
        messagebox.showwarning("Warning", "No results found. Please try different search terms")
        
    
    def search():
        xtest = scraped_db.copy()
        q = query.get()
        f = Frame(window)
        df = vertical_search_engine(xtest, q, indexed)
        if type(df) == str:
            no_result()
        else:
            pt = Table(results)
            try:
                table = pt = Table(results, dataframe=df)
                pt.show()
            except AttributeError:
                pass
            
    def close_window():
        if messagebox.askokcancel("Quit", "Quit Programme?"):
            window.destroy()
        
    
    btn = Button(window, text="Search",bg="#F0F0F0", command=clicked)
    btn.grid(column=2, row=2)
    
    

    window.protocol("WM_DELETE_WINDOW", close_window)       
    window.mainloop()

In [ ]:
new_gui(resized_image1)